In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
from keras.datasets import mnist
import tensorflow as tf
from tensorflow import keras
import math
#For the connected regions
from scipy.ndimage.measurements import label

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

THRESHOLD = 0.7
epochs = 20

def getHeight(arl):
    first = np.argmax(arl > THRESHOLD)
    tmp = np.argmax(np.flip(arl) > THRESHOLD)
    return math.ceil((arl.size - tmp - first)/arl.shape[1])/arl.shape[0]


def getWidth(arl):
    return (getHeight(np.rot90(arl))*arl.shape[0])/arl.shape[1]

structure = np.ones((3, 3), dtype=np.int) * THRESHOLD

print("Generating features")

print("Generating train data features")
for i in range(len(train_images)):
    img = train_images[i]/255
    #Get height
    height = getHeight(img)
    #Get the width
    width = getWidth(img)
    #Get the number of connected regions
    _, connected = label(img, structure)
    #Save the data in a row
    extra = np.zeros(28)
    extra[0] = height
    extra[1] = width
    if(connected == 1):
        extra[2] = 1
    elif(connected == 2):
        extra[3] = 1
    elif(connected == 3):
        extra[4] = 1
    else:
        extra[5] = 1
    #The only way for this to sort of not take forever when done this way
    train_images[i][27] = extra
    #if(i%500==0):
    #   print("{}%".format(int(100*i/len(train_images))))

print("Generating test data features")
for i in range(len(test_images)):
    img = test_images[i]/255
    #Get the heights
    height = getHeight(img)
    #Get the widths
    width = getWidth(img)
    #Get the connected parts
    _, connected = label(img, structure)
    #Save the data in a row
    extra = np.zeros(28)
    extra[0] = height
    extra[1] = width
    if(connected == 1):
        extra[2] = 1
    elif(connected == 2):
        extra[3] = 1
    elif(connected == 3):
        extra[4] = 1
    else:
        extra[5] = 1
    #The only way for this to sort of not take forever when done this way
    test_images[i][27] = extra
    #if(i%500==0):
    #    print("{}%".format(int(100*i/len(test_images))))

print("Done with features. ")

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dropout(0.15),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images,
                    train_labels,
                    epochs=epochs,
                    validation_data=(test_images, test_labels))


Generating features
Generating train data features
Generating test data features
Done with features. 
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 11s 176us/sample - loss: 2.6871 - acc: 0.8843 - val_loss: 0.4235 - val_acc: 0.9138
Epoch 2/20
60000/60000 [==============================] - 10s 164us/sample - loss: 0.4088 - acc: 0.9129 - val_loss: 0.3628 - val_acc: 0.9297
Epoch 3/20
60000/60000 [==============================] - 10s 168us/sample - loss: 0.3763 - acc: 0.9184 - val_loss: 0.3256 - val_acc: 0.9384
Epoch 4/20
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3646 - acc: 0.9214 - val_loss: 0.3418 - val_acc: 0.9355
Epoch 5/20
60000/60000 [==============================] - 9s 157us/sample - loss: 0.3336 - acc: 0.9287 - val_loss: 0.3628 - val_acc: 0.9403
Epoch 6/20
60000/60000 [==============================] - 10s 163us/sample - loss: 0.3227 - acc: 0.9314 - val_loss: 0.2895 - val_acc: 0.9418
Epo